### Note:
Script 100 creates the initial daily table. This script takes successive Mediatel datasets and then appends daily rows to the initial daily table. This way we don't have to deal with the massive stacked Mediatel dataset each time we want to update the daily metrics.

Steps:
1. Load in Mediatel data each day
2. Create metric columns
3. Summarise
4. Load current summary table
5. Append to current summary table
6. Save over current summary table

In [0]:
%run ./my_packages

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%skip
# By default mediatel dates is a list containing a single date, however, use below if need to catch up on multiple days at a time
mediatel_dates = [mediatel_date]

In [0]:
# Set `mediatel_date` here if you want to run manually. Otherwise under BAU conditions (i.e. when you don't need to run any past dates) this cell should be skipped via %skip tag.
start = '20260218'
end = '20260219'
mediatel_dates = [
    (datetime.strptime(start, "%Y%m%d") + timedelta(days=i)).strftime("%Y%m%d") for i in range((datetime.strptime(end, "%Y%m%d") - datetime.strptime(start, "%Y%m%d")).days + 1)
    ]

# Daily functions

# Weekly functions

In [0]:
for mediatel_date in mediatel_dates:
    try:
        # Read excel file
        df = f_LoadLatestMediatel(mediatel_date=mediatel_date, select_cols=select_cols)

        # Create metrics
        df = f_CreateMetrics(df, timeframe="daily")

        # Summarise
        df_summary = f_CreateSummary(df, "daily")
        df_summary = df_summary.to_pandas()

        # Load current summary table, which will be as of t-2
        table_name = "teams.cashalo.brett_collections_funnel_daily_test"
        df_current = spark.sql(f'SELECT * FROM {table_name}').toPandas()

        # Save it right back down as a back-up copy
        spark.sql(f"DROP TABLE IF EXISTS {table_name}_backup")
        spark_df_backup = spark.createDataFrame(df_current)
        spark_df_backup.write.saveAsTable(f"{table_name}_backup")

        # Append t-1 summary table to current t-2 summary table
        df_combined = pd.concat([df_current, df_summary], ignore_index=True)

        # Save over current t-2 summary table with new summary table which will be latest as of t-1
        spark_df = f_DailyCreateSparkTable(df_combined, table_name=table_name)
        spark_df.write.saveAsTable(table_name)

        display(spark_df)
            
    except Exception:
        dbutils.notebook.exit("Error encountered. Aborting script to prevent overwrite of summary data.")

call_date,DPD_type,Unique_Accounts,HC_Connect,IVR_Connect,VB_Connect,Total_Connect,HC_Connect_AgentConnect,HC_Connect_DropOff,HC_Connect_Voicemail,HC_Connect_Other,AC_PTP,AC_NO_PTP,AC_Paid_Loan,AC_DNC,AC_Call_Back,AC_Hang_Up,AC_PTP_FFUP,Processed_account,HC_Account,VB_Account,HC_Spin,VB_Spin,HC_Spin_Connect,IVR_Spin_Connect,VB_Spin_Connect,HC_Spin_Connect_DropOff,HC_Spin_Connect_Voicemail,HC_Spin_Connect_Other,HC_Spin_Connect_AgentConnect,AC_Spin_PTP,AC_Spin_NO_PTP,AC_Spin_Paid_Loan,AC_Spin_DNC,AC_Spin_Call_Back,AC_Spin_Hang_Up,AC_Spin_PTP_FFUP,VB_Only_Account
2025-11-12,M1,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,1,1,140091,158358,541,140,359,192,271,9,69,39,3,7,5,3,7,5,null
2025-11-12,Pre-Due,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,1,1,147288,156233,2347,674,1105,708,464,33,1142,922,7,123,5,7,42,36,null
2025-11-13,M1,24219,1,0,259,260,1,0,0,0,1,0,0,0,0,0,0,1,1,24219,193240,157305,547,460,328,153,289,34,71,42,0,9,5,1,8,6,null
2025-11-13,Pre-Due,22943,1,0,641,642,1,0,0,0,1,0,0,0,0,0,0,1,1,22943,185648,135667,1247,1386,895,385,347,48,467,349,6,81,2,4,12,13,null
2025-11-14,M1,24923,390,377,302,1069,56,124,189,21,30,2,15,2,2,1,4,49,24923,24290,176428,138215,496,418,366,131,286,23,56,30,2,15,2,2,1,4,null
2025-11-14,Pre-Due,22428,1037,1007,577,2621,574,226,189,48,381,3,154,5,1,6,24,544,22428,17704,137923,88555,1153,1123,666,244,284,51,574,381,3,154,5,1,6,24,null
2025-11-15,M1,20651,319,179,246,744,36,107,164,12,18,2,10,0,1,1,4,30,20651,20192,87047,123863,337,182,258,109,178,14,36,18,2,10,0,1,1,4,null
2025-11-15,Pre-Due,20565,1373,238,703,2314,894,272,160,47,458,13,248,11,4,16,144,730,20565,19685,66633,128413,1581,358,1029,322,289,56,914,458,13,251,11,5,16,160,null
2025-11-16,M1,25356,463,231,354,1048,85,134,223,21,35,3,15,5,1,1,25,58,25356,24818,106281,150150,572,247,423,143,321,23,85,35,3,15,5,1,1,25,null
2025-11-16,Pre-Due,9330,720,118,361,1199,415,175,102,28,304,4,62,2,0,18,25,372,9330,8882,35950,56680,848,190,535,204,196,33,415,304,4,62,2,0,18,25,null


call_date,DPD_type,Unique_Accounts,HC_Connect,IVR_Connect,VB_Connect,Total_Connect,HC_Connect_AgentConnect,HC_Connect_DropOff,HC_Connect_Voicemail,HC_Connect_Other,AC_PTP,AC_NO_PTP,AC_Paid_Loan,AC_DNC,AC_Call_Back,AC_Hang_Up,AC_PTP_FFUP,Processed_account,HC_Account,VB_Account,HC_Spin,VB_Spin,HC_Spin_Connect,IVR_Spin_Connect,VB_Spin_Connect,HC_Spin_Connect_DropOff,HC_Spin_Connect_Voicemail,HC_Spin_Connect_Other,HC_Spin_Connect_AgentConnect,AC_Spin_PTP,AC_Spin_NO_PTP,AC_Spin_Paid_Loan,AC_Spin_DNC,AC_Spin_Call_Back,AC_Spin_Hang_Up,AC_Spin_PTP_FFUP,VB_Only_Account
2025-11-12,M1,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,1,1,140091,158358,541,140,359,192,271,9,69,39,3,7,5,3,7,5,null
2025-11-12,Pre-Due,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,1,1,147288,156233,2347,674,1105,708,464,33,1142,922,7,123,5,7,42,36,null
2025-11-13,M1,24219,1,0,259,260,1,0,0,0,1,0,0,0,0,0,0,1,1,24219,193240,157305,547,460,328,153,289,34,71,42,0,9,5,1,8,6,null
2025-11-13,Pre-Due,22943,1,0,641,642,1,0,0,0,1,0,0,0,0,0,0,1,1,22943,185648,135667,1247,1386,895,385,347,48,467,349,6,81,2,4,12,13,null
2025-11-14,M1,24923,390,377,302,1069,56,124,189,21,30,2,15,2,2,1,4,49,24923,24290,176428,138215,496,418,366,131,286,23,56,30,2,15,2,2,1,4,null
2025-11-14,Pre-Due,22428,1037,1007,577,2621,574,226,189,48,381,3,154,5,1,6,24,544,22428,17704,137923,88555,1153,1123,666,244,284,51,574,381,3,154,5,1,6,24,null
2025-11-15,M1,20651,319,179,246,744,36,107,164,12,18,2,10,0,1,1,4,30,20651,20192,87047,123863,337,182,258,109,178,14,36,18,2,10,0,1,1,4,null
2025-11-15,Pre-Due,20565,1373,238,703,2314,894,272,160,47,458,13,248,11,4,16,144,730,20565,19685,66633,128413,1581,358,1029,322,289,56,914,458,13,251,11,5,16,160,null
2025-11-16,M1,25356,463,231,354,1048,85,134,223,21,35,3,15,5,1,1,25,58,25356,24818,106281,150150,572,247,423,143,321,23,85,35,3,15,5,1,1,25,null
2025-11-16,Pre-Due,9330,720,118,361,1199,415,175,102,28,304,4,62,2,0,18,25,372,9330,8882,35950,56680,848,190,535,204,196,33,415,304,4,62,2,0,18,25,null


# Monthly functions